In [1]:
import pandas as pd
import csv
import os
import sys
import csv
import time
import numpy as np
import random
from scipy.io import arff

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sys.path.insert(0, f"{os.path.dirname(os.getcwd())}/src")
from utils import evaluate_result
from data_imbalance_src.smote_oversampling import RandomOversampling, ADASYNOversampling, BorderlineSMOTEOversampling, SMOTEOversampling, SVMSMOTEOversampling
from data_imbalance_src.smote_oversampling import SMOTUNEDOversampling
from data_imbalance_src.dazzle import DAZZLEOversampling
from data_imbalance_src.dazzle1 import DAZZLE1Oversampling
from data_imbalance_src.dazzle2 import DAZZLE2Oversampling
from data_imbalance_src.Imbalance_Farou2022.data_generation import GANOversampling
from data_imbalance_src.random_projection import RandomProjectionOversampling

from diveplane.utilities import infer_feature_attributes
from diveplane.geminai import Geminai

from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.ModelInspector import ModelInspector
from DataSynthesizer.lib.utils import read_json_file, display_bayesian_network

from sdv.metadata import SingleTableMetadata
from sdv.lite import SingleTablePreset
from sdv.single_table import CTGANSynthesizer
from sdv.single_table import GaussianCopulaSynthesizer

2023-10-06 01:44:44.870398: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-06 01:44:44.890315: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 01:44:45.128417: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 01:44:45.132203: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 01:44:46.033340: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
rs_list = random.sample(range(50, 500), 10)

In [3]:
for repeat in range(10):
    print(f"----- in repeat {repeat+1} -----")
    rs = rs_list[repeat]
    write_file = f"Ambari_Vuln_res_r{repeat+1}_rn{rs}.csv"
    write_path = f"{os.path.dirname(os.getcwd())}/result/Ambari_Vuln/{write_file}"
    with open(write_path, "w", newline="") as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(["oversampling_scheme", "runtime", "learner", "acc", "prec", "recall", "fpr", "f1", "auc", "g_score", "d2h"])
    
    train_data_path = f"{os.path.dirname(os.getcwd())}/data/Bug_Reports/ambari-train.csv"
    test_data_path = f"{os.path.dirname(os.getcwd())}/data/Bug_Reports/ambari-test.csv"
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    
    X_train = train_df.iloc[:, :-1]
    y_train = train_df.iloc[:, -1]
    X_test = test_df.iloc[:, :-1]
    y_test = test_df.iloc[:, -1]
    print("--- y train classes count: \n" + str(y_train.value_counts()))
    print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
    print(" ")
    print("--- y test classes count: \n" + str(y_test.value_counts()))
    print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))
    
    ########## normal run ##########
    # normal run - without any oversampling technique
    print("----- normal -----")

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train)
    clf_KNN.fit(X_train_scale, y_train)
    clf_LR.fit(X_train_scale, y_train)
    clf_DT.fit(X_train_scale, y_train)
    clf_RF.fit(X_train_scale, y_train)
    clf_LightGBM.fit(X_train_scale, y_train)
    clf_Adaboost.fit(X_train_scale, y_train)
    clf_GBDT.fit(X_train_scale, y_train)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["No", 0, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["No", 0, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["No", 0, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["No", 0, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["No", 0, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["No", 0, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["No", 0, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["No", 0, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## random run ##########
    # random oversampling run - random oversampling technique
    print("----- random -----")

    rt, X_train_new, y_train_new = RandomOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["Random", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["Random", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["Random", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["Random", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["Random", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["Random", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["Random", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["Random", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## ADASYN run ##########
    # ADASYN oversampling run - ADASYN oversampling technique
    print("----- ADASYN ------")

    rt, X_train_new, y_train_new = ADASYNOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["ADASYN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["ADASYN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["ADASYN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["ADASYN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["ADASYN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["ADASYN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["ADASYN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["ADASYN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## BorderlineSMOTE run ##########
    # BorderlineSMOTE oversampling run - BorderlineSMOTE oversampling technique
    print("----- borderlineSMOTE -----")

    rt, X_train_new, y_train_new = BorderlineSMOTEOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["BorderlineSMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SMOTE run ##########
    # SMOTE oversampling run - SMOTE oversampling technique
    print("----- SMOTE -----")

    rt, X_train_new, y_train_new = SMOTEOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SVMSMOTE run ##########
    # SVMSMOTE oversampling run - SVMSMOTE oversampling technique
    print("----- SVMSMOTE -----")

    rt, X_train_new, y_train_new = SVMSMOTEOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SVMSMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SMOTUNED run ##########
    # SMOTUNED oversampling run - SMOTUNED oversampling technique
    print("----- SMOTUNED -----")

    rt_SVM, X_train_new_SVM, y_train_new_SVM = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                    y_train=y_train, y_test=y_test, model="SVM")

    scaler = StandardScaler()
    X_train_scale_SVM = pd.DataFrame(scaler.fit_transform(X_train_new_SVM), columns=X_train_new_SVM.columns)
    X_test_scale_SVM = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of SVM: 1:" + str(round(y_train_new_SVM.value_counts()[0] / y_train_new_SVM.value_counts()[1])))

    rt_KNN, X_train_new_KNN, y_train_new_KNN = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                    y_train=y_train, y_test=y_test, model="KNN")

    scaler = StandardScaler()
    X_train_scale_KNN = pd.DataFrame(scaler.fit_transform(X_train_new_KNN), columns=X_train_new_KNN.columns)
    X_test_scale_KNN = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of KNN: 1:" + str(round(y_train_new_KNN.value_counts()[0] / y_train_new_KNN.value_counts()[1])))

    rt_LR, X_train_new_LR, y_train_new_LR = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                 y_train=y_train, y_test=y_test, model="LR")

    scaler = StandardScaler()
    X_train_scale_LR = pd.DataFrame(scaler.fit_transform(X_train_new_LR), columns=X_train_new_LR.columns)
    X_test_scale_LR = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of LR: 1:" + str(round(y_train_new_LR.value_counts()[0] / y_train_new_LR.value_counts()[1])))

    rt_DT, X_train_new_DT, y_train_new_DT = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                 y_train=y_train, y_test=y_test, model="DT")

    scaler = StandardScaler()
    X_train_scale_DT = pd.DataFrame(scaler.fit_transform(X_train_new_DT), columns=X_train_new_DT.columns)
    X_test_scale_DT = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of DT: 1:" + str(round(y_train_new_DT.value_counts()[0] / y_train_new_DT.value_counts()[1])))

    rt_RF, X_train_new_RF, y_train_new_RF = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                 y_train=y_train, y_test=y_test, model="RF")

    scaler = StandardScaler()
    X_train_scale_RF = pd.DataFrame(scaler.fit_transform(X_train_new_RF), columns=X_train_new_RF.columns)
    X_test_scale_RF = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of RF: 1:" + str(round(y_train_new_RF.value_counts()[0] / y_train_new_RF.value_counts()[1])))

    rt_LightGBM, X_train_new_LightGBM, y_train_new_LightGBM = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                                   y_train=y_train, y_test=y_test, model="LightGBM")

    scaler = StandardScaler()
    X_train_scale_LightGBM = pd.DataFrame(scaler.fit_transform(X_train_new_LightGBM), columns=X_train_new_LightGBM.columns)
    X_test_scale_LightGBM = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of LightGBM: 1:" + str(round(y_train_new_LightGBM.value_counts()[0] / y_train_new_LightGBM.value_counts()[1])))

    rt_Adaboost, X_train_new_Adaboost, y_train_new_Adaboost = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                                   y_train=y_train, y_test=y_test, model="Adaboost")

    scaler = StandardScaler()
    X_train_scale_Adaboost = pd.DataFrame(scaler.fit_transform(X_train_new_Adaboost), columns=X_train_new_Adaboost.columns)
    X_test_scale_Adaboost = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of Adaboost: 1:" + str(round(y_train_new_Adaboost.value_counts()[0] / y_train_new_Adaboost.value_counts()[1])))

    rt_GBDT, X_train_new_GBDT, y_train_new_GBDT = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                       y_train=y_train, y_test=y_test, model="GBDT")

    scaler = StandardScaler()
    X_train_scale_GBDT = pd.DataFrame(scaler.fit_transform(X_train_new_GBDT), columns=X_train_new_GBDT.columns)
    X_test_scale_GBDT = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of GBDT: 1:" + str(round(y_train_new_GBDT.value_counts()[0] / y_train_new_GBDT.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale_SVM, y_train_new_SVM)
    clf_KNN.fit(X_train_scale_KNN, y_train_new_KNN)
    clf_LR.fit(X_train_scale_LR, y_train_new_LR)
    clf_DT.fit(X_train_scale_DT, y_train_new_DT)
    clf_RF.fit(X_train_scale_RF, y_train_new_RF)
    clf_LightGBM.fit(X_train_scale_LightGBM, y_train_new_LightGBM)
    clf_Adaboost.fit(X_train_scale_Adaboost, y_train_new_Adaboost)
    clf_GBDT.fit(X_train_scale_GBDT, y_train_new_GBDT)

    y_pred_SVM = clf_SVM.predict(X_test_scale_SVM)
    y_pred_KNN = clf_KNN.predict(X_test_scale_KNN)
    y_pred_LR = clf_LR.predict(X_test_scale_LR)
    y_pred_DT = clf_DT.predict(X_test_scale_DT)
    y_pred_RF = clf_RF.predict(X_test_scale_RF)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale_LightGBM)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale_Adaboost)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale_GBDT)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SMOTUNED", rt_SVM, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SMOTUNED", rt_KNN, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SMOTUNED", rt_LR, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SMOTUNED", rt_DT, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SMOTUNED", rt_RF, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SMOTUNED", rt_LightGBM, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SMOTUNED", rt_Adaboost, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SMOTUNED", rt_GBDT, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE1 -----")
    cols = X_train.columns
    tar = y_train.name

    X_train_GAN = X_train.copy()
    y_train_GAN = y_train.copy()
    X_test_GAN = X_test.copy()
    y_test_GAN = y_test.copy()
    rt, X_train_new, y_train_new, X_test_scale = DAZZLE1Oversampling(X_train=X_train_GAN, 
                                                                     y_train=y_train_GAN, 
                                                                     X_test=X_test_GAN,
                                                                     y_test=y_test_GAN)

    X_train_new = pd.DataFrame(X_train_new, columns=cols)
    y_train_new = pd.Series(y_train_new, name=tar)
    X_test_scale = pd.DataFrame(X_test_scale, columns=cols)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_new, y_train_new)
    clf_KNN.fit(X_train_new, y_train_new)
    clf_LR.fit(X_train_new, y_train_new)
    clf_DT.fit(X_train_new, y_train_new)
    clf_RF.fit(X_train_new, y_train_new)
    clf_LightGBM.fit(X_train_new, y_train_new)
    clf_Adaboost.fit(X_train_new, y_train_new)
    clf_GBDT.fit(X_train_new, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE1", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
        
    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE2 -----")
    cols = X_train.columns
    tar = y_train.name

    X_train_GAN = X_train.copy()
    y_train_GAN = y_train.copy()
    X_test_GAN = X_test.copy()
    y_test_GAN = y_test.copy()
    rt, X_train_new, y_train_new, X_test_scale = DAZZLE2Oversampling(X_train=X_train_GAN, 
                                                                     y_train=y_train_GAN, 
                                                                     X_test=X_test_GAN)

    X_train_new = pd.DataFrame(X_train_new, columns=cols)
    y_train_new = pd.Series(y_train_new, name=tar)
    X_test_scale = pd.DataFrame(X_test_scale, columns=cols)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_new, y_train_new)
    clf_KNN.fit(X_train_new, y_train_new)
    clf_LR.fit(X_train_new, y_train_new)
    clf_DT.fit(X_train_new, y_train_new)
    clf_RF.fit(X_train_new, y_train_new)
    clf_LightGBM.fit(X_train_new, y_train_new)
    clf_Adaboost.fit(X_train_new, y_train_new)
    clf_GBDT.fit(X_train_new, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE2", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## WGAN run ##########
    # WGAN oversampling run - WGAN oversampling technique
    print("----- WGAN -----")

    scaler = StandardScaler()
    X_train_GAN = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    rt, X_train_new, y_train_new = GANOversampling(X_train=X_train_GAN, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["WGAN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["WGAN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["WGAN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["WGAN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["WGAN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["WGAN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["WGAN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["WGAN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))


    ########## RandomProjection run ##########
    # Random projection oversampling run - Random projection oversampling technique
    print("----- Random Projection -----")
    X_train_RP = X_train.copy()
    y_train_RP = y_train.copy()
    rt, X_train_new, y_train_new = RandomProjectionOversampling(X_train=X_train_RP, y_train=y_train_RP)
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["RP", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["RP", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["RP", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["RP", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["RP", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["RP", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["RP", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["RP", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## Diveplane 1 run ##########
    print("----- diveplane 1 -----")
    tar = y_train.name
    conditions = [{tar: 1},
                  {tar: 0}] * (int(X_train.shape[0] / 2))
    
    X_train_diveplane_1 = X_train.copy()
    y_train_diveplane_1 = y_train.copy()
    X_train_diveplane_1[tar] = y_train_diveplane_1
    
    features = infer_feature_attributes(X_train_diveplane_1)
    for f_name, f_value in features.items():
        if f_value["type"] == "nominal":
            f_value["non_sensitive"] = True

    start_time = time.time()
    g = Geminai()
    g.train(X_train_diveplane_1, features=features)

    gen_df = g.synthesize_cases(
        n_samples=len(conditions),
        case_context_values_maps=conditions,
        desired_conviction=5,
        generate_new_cases="no"
    )

    rt = time.time() - start_time

    X_train_new = gen_df.iloc[:, :-1]
    y_train_new = gen_df.iloc[:, -1]
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["Diveplane1", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["Diveplane1", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["Diveplane1", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["Diveplane1", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["Diveplane1", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["Diveplane1", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["Diveplane1", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["Diveplane1", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## Diveplane 2 run ##########
    print("----- diveplane 2 -----")
    tar = y_train.name
    n_cases_diff = y_train.value_counts()[0] - y_train.value_counts()[1]
    conditions = [{tar: 1}] * n_cases_diff
    
    X_train_diveplane_2 = X_train.copy()
    y_train_diveplane_2 = y_train.copy()
    X_train_diveplane_2[tar] = y_train_diveplane_2
    
    features = infer_feature_attributes(X_train_diveplane_2)
    for f_name, f_value in features.items():
        if f_value["type"] == "nominal":
            f_value["non_sensitive"] = True

    start_time = time.time()
    g = Geminai()
    g.train(X_train_diveplane_2, features=features)

    gen_df = g.synthesize_cases(
        n_samples=len(conditions),
        case_context_values_maps=conditions,
        desired_conviction=5,
        generate_new_cases="no"
    )

    rt = time.time() - start_time

    X_train_new = pd.concat([X_train_diveplane_2, gen_df], ignore_index=True, axis=0)
    y_train_new = X_train_new.iloc[:, -1]
    X_train_new = X_train_new.iloc[:, :-1]
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
    
    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["Diveplane2", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["Diveplane2", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["Diveplane2", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["Diveplane2", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["Diveplane2", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["Diveplane2", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["Diveplane2", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["Diveplane2", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## DS run ##########
    print("----- DS -----")
    mode = "independent_attribute_mode"

    col = X_train.columns
    tar = y_train.name
    
    X_train_DS = X_train.copy()
    y_train_DS = y_train.copy()
    X_train_DS[tar] = y_train_DS
    write_df = X_train_DS[X_train_DS[tar] == 1]
    write_df = write_df.iloc[:, :-1]
    write_df.to_csv(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_pos_df.csv", index=False)
    
    threshold = 20
    num_tuples_to_generate = int(y_train.value_counts()[0] - y_train.value_counts()[1])

    start_time = time.time()

    description_file = f"{os.path.dirname(os.getcwd())}/extra/js_vuln.json"
    describer = DataDescriber(category_threshold=threshold)
    describer.describe_dataset_in_independent_attribute_mode(
        dataset_file=f"{os.path.dirname(os.getcwd())}/extra/js_vuln_pos_df.csv"
    )
    describer.save_dataset_description_to_file(description_file)

    generator = DataGenerator()
    generator.generate_dataset_in_independent_mode(num_tuples_to_generate, description_file)
    generator.save_synthetic_data(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_syn_df.csv")

    rt = time.time() - start_time

    X_train_new = pd.read_csv(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_syn_df.csv").to_numpy()
    y_train_new = np.ones(num_tuples_to_generate)
    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DS", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DS", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DS", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DS", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DS", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DS", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DS", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DS", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SDV FASTML run ##########
    print("----- SDV FASTML -----")
    col = X_train.columns
    tar = y_train.name
    num_tuples_to_generate = int(y_train.value_counts()[0] - y_train.value_counts()[1])
    
    X_train_SDV = X_train.copy()
    y_train_SDV = y_train.copy()
    X_train_SDV[tar] = y_train_SDV
    pos_df = X_train_SDV[X_train_SDV[tar] == 1]
    pos_df = pos_df.iloc[:, :-1]

    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data=pos_df)
    
    start_time = time.time()
    syn1 = SingleTablePreset(metadata, name="FAST_ML")
    syn1.fit(data=pos_df)
    X_train_new = syn1.sample(num_rows=num_tuples_to_generate).to_numpy()

    rt = time.time() - start_time

    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = np.ones(num_tuples_to_generate)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SDV_FASTML", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SDV GC ##########
    print("----- SDV GC -----")
    start_time = time.time()
    syn2 = GaussianCopulaSynthesizer(metadata)
    syn2.fit(data=pos_df)
    X_train_new = syn2.sample(num_rows=num_tuples_to_generate).to_numpy()

    rt = time.time() - start_time

    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = np.ones(num_tuples_to_generate)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SDV_GC", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SDV_GC", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SDV_GC", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SDV_GC", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SDV_GC", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SDV_GC", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SDV_GC", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SDV_GC", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SDV GAN ##########
    print("----- SDV GAN -----")
    start_time = time.time()
    syn3 = CTGANSynthesizer(metadata)
    syn3.fit(data=pos_df)
    X_train_new = syn3.sample(num_rows=num_tuples_to_generate).to_numpy()

    rt = time.time() - start_time

    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = np.ones(num_tuples_to_generate)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SDV_GAN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

----- in repeat 1 -----
--- y train classes count: 
0    478
1     22
Name: label, dtype: int64
--- y train ratio: 1:22
 
--- y test classes count: 
0    493
1      7
Name: label, dtype: int64
--- y test ratio: 1:70
----- normal -----
----- random -----
y train ratio: 1:1
----- ADASYN ------
y train ratio: 1:1
----- borderlineSMOTE -----
y train ratio: 1:1
----- SMOTE -----
y train ratio: 1:1
----- SVMSMOTE -----
y train ratio: 1:2
----- SMOTUNED -----
y train ratio of SVM: 1:1
y train ratio of KNN: 1:1
y train ratio of LR: 1:1
y train ratio of DT: 1:1
y train ratio of RF: 1:1
y train ratio of LightGBM: 1:1
y train ratio of Adaboost: 1:1
y train ratio of GBDT: 1:1
----- DAZZLE1 -----
[    0/1048] LG:0.039 LD:-0.246 D:0.127 GP:0.075 RMSEAVG:0.301 NUM:0.301 SynTraiAuc:0.021 RFAcc:1.000                   
[  500/1048] LG:1.076 LD:1.626 D:1.737 GP:0.022 RMSEAVG:0.301 NUM:0.301 SynTraiAuc:0.021 RFAcc:1.000                    
[ 1000/1048] LG:0.614 LD:0.716 D:0.744 GP:0.006 RMSEAVG:0.301 NUM

389                                                                                                                     
[    0/2058] LG:0.360 LD:-0.370 D:-0.021 GP:0.035 RMSEAVG:0.204 NUM:0.204 SynTraiAuc:0.976 RFAcc:1.000                  
[  500/2058] LG:0.912 LD:1.011 D:1.062 GP:0.005 RMSEAVG:0.204 NUM:0.204 SynTraiAuc:0.976 RFAcc:1.000                    
[ 1000/2058] LG:0.953 LD:0.864 D:0.882 GP:0.002 RMSEAVG:0.204 NUM:0.204 SynTraiAuc:0.976 RFAcc:1.000                    
[ 1500/2058] LG:1.004 LD:0.746 D:0.756 GP:0.001 RMSEAVG:0.073 NUM:0.073 SynTraiAuc:0.976 RFAcc:1.000                    
[ 2000/2058] LG:1.040 LD:0.674 D:0.700 GP:0.003 RMSEAVG:0.073 NUM:0.073 SynTraiAuc:0.976 RFAcc:1.000                    
[ 2058/2058] LG:1.057 LD:0.587 D:0.604 GP:0.002 RMSEAVG:0.073 NUM:0.073 SynTraiAuc:0.976 RFAcc:1.000                    
needed sample                                                                                                           
389                             

2023-10-06 02:17:07.847798: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-06 02:17:07.848705: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/150 completed. Gen loss: -0.01875651441514492. Desc loss_real: 0.03131449595093727. Desc loss_fake: 0.01875651441514492
Epoch 51/150 completed. Gen loss: -0.12709473073482513. Desc loss_real: 0.13733088970184326. Desc loss_fake: 0.12709473073482513
Epoch 101/150 completed. Gen loss: -0.03215299919247627. Desc loss_real: 0.032629549503326416. Desc loss_fake: 0.03215299919247627


/mnt/e/Research/SyntheticData/src/data_imbalance_src/Imbalance_Farou2022/data_generation.py:119: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(synthetic_data)
/mnt/e/Research/SyntheticData/src/data_imbalance_src/Imbalance_Farou2022/data_generation.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_sample = X_sample.append(X_train)
/mnt/e/Research/SyntheticData/src/data_imbalance_src/Imbalance_Farou2022/data_generation.py:135: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_sample = X_sample.drop(tar, 1)


...

y train ratio: 1:1
----- DS -----
y train ratio: 1:1
----- SDV FASTML -----
y train ratio: 1:1
----- SDV GC -----
y train ratio: 1:1
----- SDV GAN -----


/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missi

/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missi

/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missi

y train ratio: 1:1
